<a href="https://colab.research.google.com/github/TamerKaratekin/healthcare/blob/main/Evidence_Based_Medical_RAG_Assistant_(Guideline_Prototype).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Prototype: Evidence-Based RAG for Clinical Guidelines

**Author:** Tamer Karatekin  
**Objective:**  
Build a **Retrieval-Augmented Generation (RAG)** prototype that answers clinical questions based *strictly* on approved medical protocols (e.g., Sepsis Guidelines), effectively eliminating AI hallucinations.

**Strategic Context:**  
Generic Large Language Models (LLMs) like ChatGPT are unsafe for direct clinical advice because they can fabricate dosage or procedures.
In a hospital setting, **Clinical Governance** requires that AI assistants act as search engines for the "Source of Truth" (PDF Guidelines) rather than creative writers. This project demonstrates the architecture for a **Safe GenAI Assistant** that retrieves exact context before generating an answer, ensuring adherence to the Standard of Care.

**Tech Stack:**
*   **Orchestration:** `LangChain` (The industry standard for LLM flows).
*   **Vector Database:** `ChromaDB` (For semantic search).
*   **Embeddings:** `HuggingFace` (Transforming medical text into vectors).
*   **Data Source:** Simulated 2025 Sepsis Management Protocol.

Cell 1: **Install GenAI Stack**

In [ ]:
# LangChain is the industry standard for building LLM apps (Updated for Latest LangChain v0.3+)
!pip install -q langchain langchain-community langchain-core langchain-text-splitters langchain-huggingface langchain-chroma chromadb sentence-transformers

import os

# 1. Text Splitter (New Location)
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 2. Embeddings (New Location)
from langchain_huggingface import HuggingFaceEmbeddings

# 3. Vector Store (New Location)
from langchain_chroma import Chroma

# 4. Document Schema (The fix for your specific error)
from langchain_core.documents import Document

print("✅ GenAI Environment Configured.")

✅ GenAI Environment Configured.


**Cell 2: Ingest "Clinical Guidelines" (Simulated)**

In [ ]:
# PM Scenario: Doctors waste hours looking up protocol PDFs.
# We turn those PDFs into a queryable knowledge base.

guideline_text = """
GUIDELINE TITLE: Management of Sepsis (2025 Protocol)
1. INITIAL RESUSCITATION: For sepsis-induced hypoperfusion, administer at least 30mL/kg of IV crystalloid fluid within the first 3 hours.
2. ANTIMICROBIAL THERAPY: Administer IV antimicrobials within 1 hour of recognition of sepsis or septic shock.
3. VASOPRESSORS: Apply norepinephrine as the first-line agent to maintain MAP > 65 mmHg.
4. STEROIDS: IV corticosteroids are suggested only if hemodynamics cannot be restored with fluids and vasopressors.
"""

# Chunk the text (Crucial for RAG)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = [Document(page_content=x) for x in text_splitter.split_text(guideline_text)]

**Cell 3: The Brain (Vector Database)**

In [ ]:
# This cell creates the 'db' variable that was missing!

# 1. Initialize the Embedding Model (converts text to numbers)
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Create the Vector Database
db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_function
)

print("🧠 Knowledge Base Vectorized & Stored in ChromaDB.")

**# Cell 4: The Retrieval Logic (No Hallucinations)**

In [ ]:
# PM Insight: We search the DB first, then answer.

def ask_guideline_bot(query):
    # 1. Search the Vector DB for relevant chunks
    results = db.similarity_search(query, k=2)

    # 2. Construct the "Evidence"
    context = "\n".join([doc.page_content for doc in results])

    # 3. Formulate the answer (Simulating LLM generation for GitHub demo)
    # In production, this context goes to GPT-4 via API.
    print(f"❓ USER QUERY: {query}")
    print(f"🔎 RETRIEVED CONTEXT (Source of Truth):")
    print(f"   '{context}'...")
    print("-" * 30)

    if "30mL/kg" in context:
        return "Based on the 2025 Protocol, administer 30mL/kg of IV crystalloid fluid within the first 3 hours."
    elif "norepinephrine" in context:
        return "Use norepinephrine as the first-line vasopressor to maintain MAP > 65 mmHg."
    else:
        return "I cannot find that in the approved guidelines."

**# Cell 5: Test the Product**

In [ ]:
response = ask_guideline_bot("How much fluid for initial resuscitation?")
print(f"🤖 AI ANSWER: {response}")

❓ USER QUERY: How much fluid for initial resuscitation?
🔎 RETRIEVED CONTEXT (Source of Truth):
   'GUIDELINE TITLE: Management of Sepsis (2025 Protocol)
1. INITIAL RESUSCITATION: For sepsis-induced hypoperfusion, administer at least 30mL/kg of IV crystalloid fluid within the first 3 hours.
2. ANTIMICROBIAL THERAPY: Administer IV antimicrobials within 1 hour of recognition of sepsis or septic shock.
3. VASOPRESSORS: Apply norepinephrine as the first-line agent to maintain MAP > 65 mmHg.'...
------------------------------
🤖 AI ANSWER: Based on the 2025 Protocol, administer 30mL/kg of IV crystalloid fluid within the first 3 hours.


This RAG architecture solves the 'Black Box' problem of LLMs. By forcing the model to retrieve context from a curated Vector Database (Chroma) before answering, we ensure that clinical advice is grounded in approved hospital protocols, not training data hallucinations.